In [145]:
import geopandas as gpd
import pandas as pd
from h3 import h3
from geomet import wkt
import json
from ast import literal_eval
from shapely.geometry import Polygon
import contextily as ctx
#import cenpy

from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear
import folium

import json
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np

import statistics

from geojson.feature import *
from area import area

import copy

import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('max_colwidth', 200)
pd.set_option('display.max_rows', 500)

In [144]:
#!pip install geojson area

  Created wheel for area: filename=area-1.1.1-py3-none-any.whl size=3608 sha256=7661c5e32783efa38bdba7088ebc25fc7210f3bf2a2ffd5bb136cd43a441f704
  Stored in directory: /tmp/pip-ephem-wheel-cache-gx_ycfl4/wheels/59/e8/a2/0a9be8239358f5e3b48e1b5cb756f98b0f992b881e1b8b21b2
Successfully built area


In [3]:
#!pip install -U pyproj

     |████████████████████████████████| 6.4 MB 4.1 MB/s eta 0:00:01
  Attempting uninstall: pyproj
    Found existing installation: pyproj 2.6.1.post1
    Uninstalling pyproj-2.6.1.post1:
      Successfully uninstalled pyproj-2.6.1.post1


In [2]:
gpd.__version__

'0.8.1'

In [2]:
#!pip install cenpy

In [3]:
!ls /data/shape_files/DIVISA_DE_BAIRROS/DIVISA_DE_BAIRROS.shp

/data/shape_files/DIVISA_DE_BAIRROS/DIVISA_DE_BAIRROS.shp


In [38]:
!ls /data/neo4j/bus_stops/part-00000-4c2b345c-d186-435a-90f8-ead08a804687-c000.csv

/data/neo4j/bus_stops/part-00000-4c2b345c-d186-435a-90f8-ead08a804687-c000.csv


---

### BUS STOPS

In [39]:
bus_stops = pd.read_csv('/data/neo4j/bus_stops/part-00000-4c2b345c-d186-435a-90f8-ead08a804687-c000.csv')

In [40]:
bus_stops.head()

,line_code,line_way,number,name,seq,latitude,longitude,type,year,month,day
0,474,Jd. Itibere,140610,"Av. Comendador Franco, 5350 - Uberaba",20,-25.477737,-49.225416,Novo mobiliario,2019,3,10
1,060,Terminal Pinheirinho,180212,"Av. Juscelino Kubitschek de Oliveira,9008 - Cidade Industrial",22,-25.512812,-49.337605,Domus,2019,3,10
2,609,Praca Rui Barbosa,180074,"Rua Pedro Gusso, 4150 - Cidade Industrial",2,-25.511477,-49.328306,Novo mobiliario,2019,3,10
3,948,Butiatuvinha,106008,Terminal Santa Felicidade - 913 - Butiatuvinha,1,-25.401231,-49.329875,Plataforma,2019,3,10
4,706,Praca Rui Barbosa (Via Amadeu Amaral) - Domingo,180637,"Avenida Presidente Getulio Vargas, 4557 - Reboucas",25,-25.456753,-49.302465,Novo mobiliario,2019,3,10


In [133]:
h3_level = 10
 
def lat_lng_to_h3(row):
    return h3.geo_to_h3(
      float(row.latitude), float(row.longitude), h3_level)
 
bus_stops['h3'] = bus_stops.apply(lat_lng_to_h3, axis=1)

In [134]:
bus_stops.head()

,line_code,line_way,number,name,seq,latitude,longitude,type,year,month,day,h3
0,474,Jd. Itibere,140610,"Av. Comendador Franco, 5350 - Uberaba",20,-25.477737,-49.225416,Novo mobiliario,2019,3,10,8aa8313664e7fff
1,060,Terminal Pinheirinho,180212,"Av. Juscelino Kubitschek de Oliveira,9008 - Cidade Industrial",22,-25.512812,-49.337605,Domus,2019,3,10,8aa804db3d57fff
2,609,Praca Rui Barbosa,180074,"Rua Pedro Gusso, 4150 - Cidade Industrial",2,-25.511477,-49.328306,Novo mobiliario,2019,3,10,8aa804db2357fff
3,948,Butiatuvinha,106008,Terminal Santa Felicidade - 913 - Butiatuvinha,1,-25.401231,-49.329875,Plataforma,2019,3,10,8aa804cb4557fff
4,706,Praca Rui Barbosa (Via Amadeu Amaral) - Domingo,180637,"Avenida Presidente Getulio Vargas, 4557 - Reboucas",25,-25.456753,-49.302465,Novo mobiliario,2019,3,10,8aa804d922d7fff


In [135]:
def counts_by_hexagon(df, resolution):
    
    '''Use h3.geo_to_h3 to index each data point into the spatial index of the specified resolution.
      Use h3.h3_to_geo_boundary to obtain the geometries of these hexagons'''

    df = df[["latitude","longitude"]]
    
    df["hex_id"] = df.apply(lambda row: h3.geo_to_h3(row["latitude"], row["longitude"], resolution), axis = 1)
    
    df_aggreg = df.groupby(by = "hex_id").size().reset_index()
    df_aggreg.columns = ["hex_id", "value"]
    
    df_aggreg["geometry"] =  df_aggreg.hex_id.apply(lambda x: 
                                                           {    "type" : "Polygon",
                                                                 "coordinates": 
                                                                [h3.h3_to_geo_boundary(x,geo_json=True)]
                                                            }
                                                        )
    
    return df_aggreg

In [137]:
df_aggreg = counts_by_hexagon(bus_stops,9)
df_aggreg.sort_values(by = "value", ascending = False, inplace = True)
df_aggreg.head(5)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,hex_id,value,geometry
2037,89a831ac9c7ffff,132,"{'type': 'Polygon', 'coordinates': [((-49.29504721592801, -25.513925698887082), (-49.29398345862807, -25.512504811906368), (-49.294944371743306, -25.51095035652311), (-49.296969023547724, -25.5108..."
1715,89a83136e63ffff,115,"{'type': 'Polygon', 'coordinates': [((-49.27085917908301, -25.4304796743032), (-49.26979551448398, -25.429056767184633), (-49.270756457026515, -25.427500830279108), (-49.27278104560325, -25.427367..."
250,89a804d92d3ffff,97,"{'type': 'Polygon', 'coordinates': [((-49.29072487603997, -25.476925446154812), (-49.28966117401724, -25.475503728387096), (-49.2906220574226, -25.47394859510214), (-49.29264662424951, -25.4738151..."
400,89a804d96c3ffff,96,"{'type': 'Polygon', 'coordinates': [((-49.29134180797536, -25.494784041307895), (-49.29027807601443, -25.493362709323726), (-49.29123898357759, -25.49180790777479), (-49.29326360449836, -25.491674..."
2165,89a831acd63ffff,94,"{'type': 'Polygon', 'coordinates': [((-49.29535575426392, -25.52285100790229), (-49.294291981994384, -25.521430313943434), (-49.2952529071883, -25.519876024585514), (-49.29727758603994, -25.519742..."


In [146]:
def hexagons_dataframe_to_geojson(df_hex, file_output = None):
    
    '''Produce the GeoJSON for a dataframe that has a geometry column in geojson format already, along with the columns hex_id and value '''
    
    list_features = []
    
    for i,row in df_hex.iterrows():
        feature = Feature(geometry = row["geometry"] , id=row["hex_id"], properties = {"value" : row["value"]})
        list_features.append(feature)
        
    feat_collection = FeatureCollection(list_features)
    
    geojson_result = json.dumps(feat_collection)
    
    #optionally write to file
    if file_output is not None:
        with open(file_output,"w") as f:
            json.dump(feat_collection,f)
    
    return geojson_result

In [147]:
def choropleth_map(df_aggreg, border_color = 'black', fill_opacity = 0.7, initial_map = None, with_legend = False,
                   kind = "linear"):
    #colormap
    min_value = df_aggreg["value"].min()
    max_value = df_aggreg["value"].max()
    m = round ((min_value + max_value ) / 2 , 0)
    
    #take resolution from the first row
    res = h3.h3_get_resolution(df_aggreg.loc[0,'hex_id'])
    
    if initial_map is None:
        initial_map = Map(location= [43.600378, 1.445478], zoom_start=11, tiles="cartodbpositron", 
                attr= '© <a href="http://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors © <a href="http://cartodb.com/attributions#basemaps">CartoDB</a>' 
            )
        

    #the colormap 
    #color names accepted https://github.com/python-visualization/branca/blob/master/branca/_cnames.json
    if kind == "linear":
        custom_cm = cm.LinearColormap(['green','yellow','red'], vmin=min_value, vmax=max_value)
    elif kind == "outlier":
        #for outliers, values would be -11,0,1
        custom_cm = cm.LinearColormap(['blue','white','red'], vmin=min_value, vmax=max_value)
    elif kind == "filled_nulls":
        custom_cm = cm.LinearColormap(['sienna','green','yellow','red'], 
                                      index=[0,min_value,m,max_value],vmin=min_value,vmax=max_value)
   

    #create geojson data from dataframe
    geojson_data = hexagons_dataframe_to_geojson(df_hex = df_aggreg)
    
    #plot on map
    name_layer = "Choropleth " + str(res)
    if kind != "linear":
        name_layer = name_layer + kind
        
    GeoJson(
        geojson_data,
        style_function=lambda feature: {
            'fillColor': custom_cm(feature['properties']['value']),
            'color': border_color,
            'weight': 1,
            'fillOpacity': fill_opacity 
        }, 
        name = name_layer
    ).add_to(initial_map)

    #add legend (not recommended if multiple layers)
    if with_legend == True:
        custom_cm.add_to(initial_map)
    
    
    
    return initial_map

In [149]:
m_hex = choropleth_map(df_aggreg = df_aggreg, with_legend = True)

m_hex.save('3_choropleth_counts_by_hexid9.html')
m_hex

### BAIRROS

In [177]:
shapefile = '/data/shape_files/DIVISA_DE_BAIRROS/DIVISA_DE_BAIRROS.shp'

bairros = gpd.read_file(shapefile,crs="epsg:29192")
#bairros.crs = "EPSG:4326"

In [178]:
bairros.crs

<Projected CRS: EPSG:29192>
Name: SAD69 / UTM zone 22S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - onshore and offshore northern and southern hemispheres between 54°W and 48°W. In remainder of South America - between 54°W and 48°W, southern hemisphere, onshore.
- bounds: (-54.0, -35.71, -47.99, 7.04)
Coordinate Operation:
- name: UTM zone 22S
- method: Transverse Mercator
Datum: South American Datum 1969
- Ellipsoid: GRS 1967 Modified
- Prime Meridian: Greenwich

In [179]:
bairros = bairros.to_crs("EPSG:4326") 

In [180]:
bairros = bairros[['CODIGO','NOME','CD_REGIONA','NM_REGIONA','geometry']]

#Apply function to create new column
bairros["geojson"] = bairros["geometry"].apply(lambda x: json.dumps(wkt.loads(str(x))))

In [181]:
bairros.head(1)

,CODIGO,NOME,CD_REGIONA,NM_REGIONA,geometry,geojson
0,9.0,ÁGUA VERDE,7.0,REGIONAL PORTÃO,"POLYGON ((-49.27115 -25.45303, -49.27086 -25.45366, -49.27063 -25.45416, -49.26998 -25.45558, -49.26946 -25.45671, -49.26940 -25.45684, -49.26930 -25.45706, -49.26929 -25.45708, -49.26953 -25.4572...","{""type"": ""Polygon"", ""coordinates"": [[[-49.27114921854898, -25.45303134036689], [-49.27085924642682, -25.45366290304544], [-49.27063002808693, -25.4541621372632], [-49.26997694111615, -25.455584530..."


In [182]:
bairros['h3'] = bairros["geojson"].apply(lambda x: list(h3.polyfill(literal_eval(x),10,geo_json_conformant=True)))

In [183]:
bairros.head()

,CODIGO,NOME,CD_REGIONA,NM_REGIONA,geometry,geojson,h3
0,9.0,ÁGUA VERDE,7.0,REGIONAL PORTÃO,"POLYGON ((-49.27115 -25.45303, -49.27086 -25.45366, -49.27063 -25.45416, -49.26998 -25.45558, -49.26946 -25.45671, -49.26940 -25.45684, -49.26930 -25.45706, -49.26929 -25.45708, -49.26953 -25.4572...","{""type"": ""Polygon"", ""coordinates"": [[[-49.27114921854898, -25.45303134036689], [-49.27085924642682, -25.45366290304544], [-49.27063002808693, -25.4541621372632], [-49.26997694111615, -25.455584530...","[8aa83136cc67fff, 8aa83136526ffff, 8aa83136e82ffff, 8aa831361857fff, 8aa83136cd57fff, 8aa831361887fff, 8aa831361b67fff, 8aa83136cd1ffff, 8aa804d92587fff, 8aa831361837fff, 8aa831361117fff, 8aa83136..."
1,18.0,JARDIM SOCIAL,1.0,REGIONAL MATRIZ,"POLYGON ((-49.23000 -25.41648, -49.23000 -25.41650, -49.22984 -25.41650, -49.22970 -25.41651, -49.22959 -25.41654, -49.22951 -25.41657, -49.22946 -25.41662, -49.22945 -25.41663, -49.22942 -25.4166...","{""type"": ""Polygon"", ""coordinates"": [[[-49.23000231238066, -25.4164766431429], [-49.22999669874449, -25.41649904981508], [-49.22984075788015, -25.41649717141305], [-49.22970335184853, -25.416512578...","[8aa831340947fff, 8aa831340ba7fff, 8aa831340387fff, 8aa83134095ffff, 8aa831340b8ffff, 8aa83134016ffff, 8aa8313401affff, 8aa83134032ffff, 8aa831340bb7fff, 8aa831340147fff, 8aa831340167fff, 8aa83134..."
2,57.0,XAXIM,2.0,REGIONAL BOQUEIRÃO,"POLYGON ((-49.26498 -25.48821, -49.26412 -25.48998, -49.26320 -25.48960, -49.26320 -25.48960, -49.26319 -25.48962, -49.26257 -25.49080, -49.26209 -25.49132, -49.26208 -25.49133, -49.26207 -25.4913...","{""type"": ""Polygon"", ""coordinates"": [[[-49.26498086210688, -25.48821073291642], [-49.26411944558158, -25.48997980078887], [-49.26319976230528, -25.48959958105909], [-49.26319889714948, -25.48960123...","[8aa831acaaaffff, 8aa83136484ffff, 8aa831acb4a7fff, 8aa831acb89ffff, 8aa831ac851ffff, 8aa831ace28ffff, 8aa831acb5b7fff, 8aa831acbc0ffff, 8aa831acb8c7fff, 8aa831aca027fff, 8aa831aca047fff, 8aa831ac..."
3,22.0,JARDIM DAS AMÉRICAS,3.0,REGIONAL CAJURU,"POLYGON ((-49.22781 -25.47203, -49.22881 -25.47264, -49.22882 -25.47264, -49.22953 -25.47165, -49.23025 -25.47068, -49.23057 -25.47024, -49.23100 -25.46965, -49.23159 -25.46884, -49.23177 -25.4686...","{""type"": ""Polygon"", ""coordinates"": [[[-49.22781211130422, -25.47203413922198], [-49.22881441221552, -25.4726380585511], [-49.22881643954093, -25.47263528829297], [-49.22953481526515, -25.471654107...","[8aa83137535ffff, 8aa83136221ffff, 8aa831362c07fff, 8aa8313620d7fff, 8aa831362027fff, 8aa8313666affff, 8aa83136238ffff, 8aa831344817fff, 8aa831362327fff, 8aa831362197fff, 8aa8313620f7fff, 8aa83136..."
4,26.0,GUAÍRA,7.0,REGIONAL PORTÃO,"POLYGON ((-49.28244 -25.47696, -49.28353 -25.47723, -49.28382 -25.47628, -49.28382 -25.47628, -49.28384 -25.47621, -49.28384 -25.47620, -49.28410 -25.47534, -49.28417 -25.47532, -49.28477 -25.4750...","{""type"": ""Polygon"", ""coordinates"": [[[-49.2824403851771, -25.47696031630578], [-49.28353256858151, -25.47722668743916], [-49.28381557901791, -25.47627967924198], [-49.28381564787584, -25.476279446...","[8aa83136575ffff, 8aa8313656b7fff, 8aa8313656cffff, 8aa831361db7fff, 8aa831360baffff, 8aa831365617fff, 8aa831360a67fff, 8aa8313654cffff, 8aa8313652d7fff, 8aa8313656a7fff, 8aa831365307fff, 8aa83136..."


In [187]:
bus_stops[bus_stops.h3.isin(bairros.h3[0]) == True]

,line_code,line_way,number,name,seq,latitude,longitude,type,year,month,day,h3
113,760,Praca Rui Barbosa,150637,"Av. Pres. Getulio Vargas, 1957 - Reboucas",17,-25.447493,-49.278395,Novo mobiliario,2019,3,10,8aa83136e927fff
159,202,Atendimento Terminal Boa Vista,108035,Estac?o Tubo Bento Viana,13,-25.444886,-49.285470,Estac?o tubo,2019,3,10,8aa83136c58ffff
169,777,Vila Velha,150632,"Av. Iguacu, 2083 - Agua Verde",7,-25.446700,-49.281392,Novo mobiliario,2019,3,10,8aa8313612dffff
196,674,Nossa Sra. da Luz,150633,"Av. Iguacu, 2436 - Agua Verde",7,-25.447884,-49.284623,Novo mobiliario,2019,3,10,8aa83136125ffff
251,762,V. Rosinha,150632,"Av. Iguacu, 2083 - Agua Verde",7,-25.446700,-49.281392,Novo mobiliario,2019,3,10,8aa8313612dffff
...,...,...,...,...,...,...,...,...,...,...,...,...
20161,789,Caiua,150631,"Av. Iguacu, 1788 - Agua Verde",5,-25.445710,-49.278719,Novo mobiliario,2019,3,10,8aa83136e90ffff
20191,706,Terminal Caiua (Via Amadeu Amaral) - Domingo,150633,"Av. Iguacu, 2436 - Agua Verde",7,-25.447884,-49.284623,Novo mobiliario,2019,3,10,8aa83136125ffff
20198,706,Praca Rui Barbosa (Via Rezala Sim?o) - Sabado,180535,"Av. Pres. Getulio Vargas, 3047 - Agua Verde",28,-25.451467,-49.288748,Novo mobiliario,2019,3,10,8aa83136cca7fff
20214,X29,Praca Rui Barbosa,108041,Estac?o Tubo Dom Pedro l,10,-25.456920,-49.288508,Estac?o tubo,2019,3,10,8aa831361a67fff


In [189]:
bairros.NOME[1]

'JARDIM SOCIAL'

In [188]:
bus_stops[bus_stops.h3.isin(bairros.h3[1]) == True]

,line_code,line_way,number,name,seq,latitude,longitude,type,year,month,day,h3
386,211,Atendimento Amancio Moro - Term.Cabral,130943,"R. Mexico, 1367 - Bacacheri",3,-25.412932,-49.234095,Chapeu chines,2019,3,10,8aa83134039ffff
821,374,Praca Santos Andrade,130270,"Rua Fagundes Varela, 101 - Jardim Social",17,-25.417415,-49.239795,Novo mobiliario,2019,3,10,8aa831340af7fff
1065,211,Terminal Cabral (Via Amancio Moro),130470,"Av. Pres. Washington Luis, 645 - Jardim Social",10,-25.419231,-49.230508,Novo mobiliario,2019,3,10,8aa831340037fff
1254,021,Terminal Cabral,130129,"Av. Nossa Sra. da Luz, 2561 - Alto da Rua XV",15,-25.425910,-49.242160,Novo mobiliario,2019,3,10,8aa8313454a7fff
1370,361,Praca Santos Andrade,130703,"Rua Cel. Rom?o Rodrigues de Oliveira Branco, 423 - Jardim Social",15,-25.411465,-49.237972,Novo mobiliario,2019,3,10,8aa831340207fff
1566,023,Terminal Campina do Siqueira,109048,Estac?o Tubo Praca das Nac?es,16,-25.425924,-49.242351,Estac?o tubo,2019,3,10,8aa8313454a7fff
1857,365,Jd. Social,130462,"Av. Pres. Washington Luis, 100 Jardim Social",29,-25.419359,-49.235984,Placa em cano,2019,3,10,8aa83134016ffff
2045,023,Terminal do Port?o,109048,Estac?o Tubo Praca das Nac?es,12,-25.425924,-49.242351,Estac?o tubo,2019,3,10,8aa8313454a7fff
2117,371,Terminal Bairro Alto,130449,"Rua Papa Rafael, 654 - Jardim Social",10,-25.425508,-49.234963,Novo mobiliario,2019,3,10,8aa831340807fff
2519,022,Horario,109048,Estac?o Tubo Praca das Nac?es,3,-25.425924,-49.242351,Estac?o tubo,2019,3,10,8aa8313454a7fff
